# Test for different multiscale approach

In our research we found out using pyramid representation although saves the storage space but theoratically loses the precision in parameters. Therefore, in this script we tried to operate the different strategies in mutli-scale parametrization, including now:

1. tune the search distance/step for diversifying scale
2. coarsen the resolution, derive the parameters, and ultimately resample back


## Part 1: Set up

In [ ]:
import os
import rasterio
import matplotlib.pyplot as plt
os.environ['USE_PYGEOS'] = '0'
import geopandas as gpd
import numpy as np
from shapely.geometry import Polygon,mapping,box
from shapely import segmentize
import time
import sys
from joblib import Parallel, delayed
from minio import Minio
from eumap.misc import ttprint
import requests
import pandas as pd
import pickle
# set up whiteboxworkflow environment
import whitebox_workflows
from whitebox_workflows import download_sample_data, show, WbEnvironment
wbe = whitebox_workflows.WbEnvironment()
import requests

In [ ]:
## scale factors for each parameters
p_table=pd.read_csv('scaling.csv')
p_table.head()

In [ ]:
#os.system(f'rm -r tmp-global-geomorpho/*')
with open('/mnt/slurm/jobs/global_geomorphometric-whiteboxtool/shuf.txt', 'r') as file:
    shuf = [int(line.strip()) for line in file]
    
with open(f'/mnt/slurm/jobs/global_geomorphometric-whiteboxtool/equi7_tiles', "rb") as fp:   # Unpickling
    args_whole = pickle.load(fp)
    
start_tile=5
end_tile=6
args = args_whole[start_tile:end_tile]

## Part 2: Run the parametrization in different approach using two examples

1. geomorphons
2. spherical_std_dev_of_normals

In [ ]:
for info in args:
    equi7_bounds_final,equi7_bounds_rls,epsg4326_bounds_rls,equi7_bounds_ls,epsg4326_bounds_ls,tile_name,equi_7_proj = info[0],info[1],info[2],info[3],info[4],info[5],info[6]


    outdir=f'/mnt/apollo/tmp-global-geomorpho/{tile_name}'
    os.makedirs(outdir,exist_ok=True)
    gdalwarp_bbox_rls = ' '.join([str(i) for i in equi7_bounds_rls])
    gdalwarp_bbox_ls = ' '.join([str(i) for i in equi7_bounds_ls])
    gdalwarp_bbox_final = ' '.join([str(i) for i in equi7_bounds_final])
    filepath = 'legendtm_rf_30m_m_s_20000101_20231231_go_epsg.4326_v20250130.tif'

    start_time = time.time()
    tmp_outdir=f'/tmp/tmp-global-geomorpho/{tile_name}'
    os.makedirs(tmp_outdir,exist_ok=True)
    gdal_cmd = f'gdalwarp  -co TILED=YES -co BIGTIFF=YES -co COMPRESS=DEFLATE \
    -co ZLEVEL=9 -co BLOCKXSIZE=1024 -co BLOCKYSIZE=1024 -co NUM_THREADS=8 \
    -co SPARSE_OK=TRUE -of GTiff -overwrite'
    out_ori_file=f'dtm_edtm_m_30m_s_20000101_20221231_{tile_name.lower().replace("_",".")}.rls_equi7_v20241230.tif'
    url_rls=f'http://{ip}/tmp-global-geomorpho/v4/{tile_name}/{out_ori_file}'

    r = requests.head(url_rls)
    if r.status_code == 200:
        ttprint(f'{url_rls} exists')
    else:
        rn_file = f'{tmp_outdir}/{out_ori_file}'
        os.system(f'{gdal_cmd} -t_srs "{equi_7_proj}" \
        -te {gdalwarp_bbox_rls} -tr 30 30 -r bilinear {filepath} {tmp_outdir}/scaled_dtm_tmp_rls.tif')
        os.system(f'gdal_calc.py --overwrite -A {tmp_outdir}/scaled_dtm_tmp_rls.tif \
        --outfile={rn_file} --calc="A * 0.1" \
        --type=Float32 --co="COMPRESS=DEFLATE" --co="BLOCKXSIZE=2048" --co="BLOCKYSIZE=2048"')
        
    for resolution in [60]:
        url=f'http://{ip}/tmp-global-geomorpho/{tile_name}/tan.curv_edtm_m_{resolution}m_s_20000101_20221231_go_epsg.4326_v20241230.tif'
 
        if resolution==30:
            tmp_dtm_rls_file = f'{tmp_outdir}/dtm_tmp_rls_{resolution}.tif'
            os.system(f'{gdal_cmd} /vsicurl/{url_rls} {tmp_dtm_rls_file}')
            # crop to local land surface tiff
            tmp_dtm_ls_file = f'{tmp_outdir}/dtm_tmp_ls.tif'
            os.system(f'{gdal_cmd} -te {gdalwarp_bbox_ls} /vsicurl/{url_rls} {tmp_dtm_ls_file}')

        else:            
            # crop to regional land surface tiff
            tmp_dtm_rls_file = f'{tmp_outdir}/dtm_tmp_rls_{resolution}.tif'
            os.system(f'{gdal_cmd} -r average -tr {resolution} {resolution} -te {gdalwarp_bbox_rls} /vsicurl/{url_rls} {tmp_dtm_rls_file}')
     
            # crop to local land surface tiff
            tmp_dtm_ls_file = f'{tmp_outdir}/dtm_tmp_ls_{resolution}.tif'
            os.system(f'{gdal_cmd} -r average -tr {resolution} {resolution} -te {gdalwarp_bbox_ls} /vsicurl/{url_rls} {tmp_dtm_ls_file}')
    
        # crop the landmask
        global_landmask_file='http://{ip}/global/dsm.landmask_ensemble_m_30m_s_20000101_20221231_go_epsg.4326_v4.1.tif'
        tmp_landmask_file = f'{tmp_outdir}/landmask_{resolution}.tif'
        os.system(f'{gdal_cmd} -t_srs "{equi_7_proj}" -r min -tr {resolution} {resolution} -te {gdalwarp_bbox_final} {global_landmask_file} {tmp_landmask_file}')


        start_time = time.time()
        # Reading raster data
        dtm = wbe.read_raster(tmp_dtm_rls_file)
        ttprint(f"{tile_name} read_raster--- %s seconds ---" % (time.time() - start_time))

        file_list=[]
        if resolution == 30:
            start_time = time.time()
            dtm = wbe.gaussian_filter(dtm)
            ttprint(f"{tile_name} calculate gaussian filter--- %s seconds ---" % (time.time() - start_time))    


        # geomorphon
        tmp_geomorphon_file=tmp_dtm_rls_file.replace('dtm','geomorphon.3')
        scale=p_table[p_table['parameters']=='geomorphon'].multiplier.iloc[0]

        start_time = time.time()
        geomorphon=wbe.geomorphons(dtm, search_distance=3, 
                                  output_forms=True, analyze_residuals=False)
        wbe.write_raster(geomorphon*scale, tmp_geomorphon_file, compress=True)#, compress=False) # Compression is good, but it is a bit slower so here we won't use it.
        ttprint(f"{tile_name} calculate geomporphon--- %s seconds ---" % (time.time() - start_time))    
        file_list.append(tmp_geomorphon_file)

        tmp_geomorphon_file=tmp_dtm_rls_file.replace('dtm','geomorphon.6')
        scale=p_table[p_table['parameters']=='geomorphon'].multiplier.iloc[0]

        start_time = time.time()
        geomorphon=wbe.geomorphons(dtm, search_distance=6, 
                                  output_forms=True, analyze_residuals=False)
        wbe.write_raster(geomorphon*scale, tmp_geomorphon_file, compress=True)#, compress=False) # Compression is good, but it is a bit slower so here we won't use it.
        ttprint(f"{tile_name} calculate geomporphon--- %s seconds ---" % (time.time() - start_time))    
        file_list.append(tmp_geomorphon_file)

        #Spherical Std Dev Of Normals
        start_time = time.time()
        tmp_ssdon_file=tmp_dtm_ls_file.replace('dtm','ssdon.3')
        scale=p_table[p_table['parameters']=='ssdon'].multiplier.iloc[0]

        start_time = time.time()
        ssdon=wbe.spherical_std_dev_of_normals(
            dtm, 
            filter_size=3 
        )

        wbe.write_raster(ssdon*scale, tmp_ssdon_file, compress=True) # Compression is good, but it is a bit slower so here we won't use it.
        ttprint(f"{tile_name} spherical std dev of normals--- %s seconds ---" % (time.time() - start_time))
        file_list.append(tmp_ssdon_file)

        #Spherical Std Dev Of Normals
        start_time = time.time()
        tmp_ssdon_file=tmp_dtm_ls_file.replace('dtm','ssdon.6')
        scale=p_table[p_table['parameters']=='ssdon'].multiplier.iloc[0]

        start_time = time.time()
        ssdon=wbe.spherical_std_dev_of_normals(
            dtm, 
            filter_size=6
        )

        wbe.write_raster(ssdon*scale, tmp_ssdon_file, compress=True) # Compression is good, but it is a bit slower so here we won't use it.
        ttprint(f"{tile_name} spherical std dev of normals--- %s seconds ---" % (time.time() - start_time))
        file_list.append(tmp_ssdon_file)

        
        start_time = time.time()
        def para_gdal_warp(file_path,tile_name,bbox,p_table,tmp_landmask_file):
            file_name = file_path.split('/')[-1]
            parameter = file_name.split('_')[0]
            dtype=p_table[p_table['parameters']==parameter.split('.')[0]].new_data_type.iloc[0]
            no_data=p_table[p_table['parameters']==parameter.split('.')[0]].no_data.iloc[0]

            gdalcmd = f'gdalwarp -overwrite -ot {dtype} -tr {resolution} {resolution} -te {bbox} -co TILED=YES -co BIGTIFF=YES -co COMPRESS=DEFLATE -co ZLEVEL=9 -co BLOCKXSIZE=2048 -co BLOCKYSIZE=2048 -co NUM_THREADS=8 -co SPARSE_OK=TRUE'

            file_name = parameter + '_edtm' + '_m' + f'_{resolution}m' + '_s' + '_20000101_20221231' + '_go'  + '_epsg.4326' + '_v20241230' + '.tif'
            out_path = f'{outdir}/{file_name}'
            tmp_out_path = f'{outdir}/tmp_{file_name}'
            os.system(f'{gdalcmd} {file_path} {tmp_out_path}')
            # landmasking
            os.system(f'gdal_calc.py -A {tmp_out_path} -B {tmp_landmask_file} --overwrite --outfile={out_path} \
                        --calc="(B==100)*A + (B!=100)*{no_data}" --type={dtype} --co="ZLEVEL=9" --co="COMPRESS=DEFLATE" \
                        --co="BLOCKXSIZE=2048" --NoDataValue={no_data} --co="BLOCKYSIZE=2048" \
                        --co="NUM_THREADS=8" --co="SPARSE_OK=TRUE"')
            os.remove(file_path)
            return out_path,file_name

        args = [(i,tile_name,gdalwarp_bbox_final,p_table,tmp_landmask_file) for i in file_list]
        for arg in args:
            out_file,rn_file=para_gdal_warp(arg[0],arg[1],arg[2],arg[3],arg[4])
            s3_path = f"{tile_name}/{rn_file}"
            #client.fput_object(s3_config['bucket'], s3_path, out_file)
            #os.remove(out_file)
            ttprint(f'http://{ip}/tmp-global-geomorpho/{s3_path} on S3')
        os.remove(tmp_dtm_ls_file)
        os.remove(tmp_dtm_rls_file)
        os.system(f'rm -r {tmp_outdir}/*')
        ttprint(f"{tile_name} crop and save to local--- %s seconds ---" % (time.time() - start_time))

#Parallel(n_jobs=10)(delayed(worker)(i,p_table) for i in args)

## Part 3: Crop and compare

In [ ]:
# generic gdal command
gdalcmd = f'gdalwarp -overwrite -r cubicspline -tr 60 60 -co TILED=YES -co BIGTIFF=YES -co COMPRESS=DEFLATE -co ZLEVEL=9 -co BLOCKXSIZE=2048 -co BLOCKYSIZE=2048 -co NUM_THREADS=8 -co SPARSE_OK=TRUE'

In [ ]:
os.system(f'{gdalcmd} test_tiles/geomorphon.3_edtm_m_120m_s_20000101_20221231_go_epsg.4326_v20241230.tiff \
test_tiles/geomorphon.r120_edtm_m_60m_s_20000101_20221231_go_epsg.4326_v20241230.tiff')

In [ ]:
os.system(f'{gdalcmd} test_tiles/ssdon.3_edtm_m_120m_s_20000101_20221231_go_epsg.4326_v20241230.tiff \
test_tiles/ssdon.r120_edtm_m_60m_s_20000101_20221231_go_epsg.4326_v20241230.tiff')

In [ ]:
sshon_r120m=rasterio.open('test_tiles/ssdon.r120_edtm_m_60m_s_20000101_20221231_go_epsg.4326_v20241230.tiff').read(1)
sshon_60m=rasterio.open('test_tiles/ssdon.6_edtm_m_60m_s_20000101_20221231_go_epsg.4326_v20241230.tiff').read(1)

In [ ]:
plt.scatter(sshon_60m,sshon_r120m)